### Production Features Pipeline

This notebook is run daily from a Github Action. 

1. It scrapes the results from the previous day's games, performs feature engineering, and saves the results back to the Feature Store at Hopsworks.ai

2. It scrapes the upcoming games for today, and saves the blank records back into the Feature Store at Hopsworks.ai so that they can be accessed by the model for the prediction service via the streamlit app.

**Note:**
There are two options for webscraping in this notebook. 
Set the 'WEBSCRAPER' variable to either 'SCRAPINGANT' or 'SELENIUM' to choose which version to run.

1. SCRAPINGANT: Uses a webscraping service with a Python API, ScrapingAnt, which handles all the proxy server issues, but does require an account. The free account allows for 1000 page requests, which is more than enough for this project. Proxies are required when running this notebook from a Github Action or otherwise key data will fail to be scraped from NBA.com. 

2. SELENIUM: This option does not currently integrate proxy servers into the webscraping process, which can cause issues when scraping from certain locations, in particular Github Actions. 

In [1]:
# select web scraper; 'SCRAPINGANT' or 'SELENIUM'
# SCRAPINGANT requires a subscription but includes a proxy server

WEBSCRAPER = 'SCRAPINGANT'
#WEBSCRAPER = 'SELENIUM'

In [2]:
import os

import pandas as pd
import numpy as np

import hopsworks

from datetime import datetime, timedelta
from pytz import timezone

import json

import time

from pathlib import Path  #for Windows/Linux compatibility

# change working directory to project root when running from notebooks folder to make it easier to import modules
# and to access sibling folders
os.chdir('..') 

 
from src.webscraping import (
    get_new_games,
    activate_web_driver,
    get_todays_matchups,
)

from src.data_processing import (
    process_games,
    add_TARGET,
)

from src.feature_engineering import (
    process_features,
)

from src.hopsworks_utils import (
    save_feature_names,
    convert_feature_names,
)


DATAPATH = Path(r'data')

**Load API keys**

In [3]:
from dotenv import load_dotenv

load_dotenv()

try:
    HOPSWORKS_API_KEY = os.environ['HOPSWORKS_API_KEY']
except:
    raise Exception('Set environment variable HOPSWORKS_API_KEY')

# if scrapingant is chosen then set the api key, otherwise load the selenium webdriver
if WEBSCRAPER == 'SCRAPINGANT':
    try:
        SCRAPINGANT_API_KEY = os.environ['SCRAPINGANT_API_KEY']
    except:
        raise Exception('Set environment variable SCRAPINGANT_API_KEY')
    driver = None
    
elif WEBSCRAPER == 'SELENIUM':
    driver = activate_web_driver('chromium')
    SCRAPINGANT_API_KEY = ""
    



**Scrape New Completed Games and Format Them**

In [4]:


df_new = get_new_games(SCRAPINGANT_API_KEY, driver)

if df_new.empty:
    print('No new games to process')
else:

    # get the SEASON of the last game in the database
    # this will used when constructing rows for prediction
    SEASON = df_new['SEASON'].max()

    df_new




Current month is 05
Scraping https://www.nba.com/stats/teams/boxscores?SeasonType=Playoffs&DateFrom=04/29/23&DateTo=05/01/23


**Retrieve todays games**

In [5]:
#retrieve list of teams playing today

# get today's games on NBA schedule
matchups, game_ids = get_todays_matchups(SCRAPINGANT_API_KEY, driver)

if matchups is None:
    print('No games today')
else:
    print(matchups)
    print(game_ids)


Mon
[['1610612755', '1610612738'], ['1610612756', '1610612743']]
['42200211', '42200222']


**Close Webdriver**

In [6]:
if WEBSCRAPER == 'SELENIUM':
    driver.close() 

**Check if anything is going on in the season**

In [7]:
if (df_new.empty) and (matchups is None):
    print('No new games to process')
    exit()
    

**Create Rows for Today's Games with Empty Stats**

In [8]:
# reformat today's matchups to the new games dataframe

if matchups is None:
    print('No games going on. Nothing to do.')
    exit()    

else:

    df_today = df_new.drop(df_new.index) #empty copy of df_new with same columns
    for i, matchup in enumerate(matchups):
        game_details = {'HOME_TEAM_ID': matchup[1], 
                        'VISITOR_TEAM_ID': matchup[0], 
                        'GAME_DATE_EST': datetime.now(timezone('EST')).strftime("%Y-%m-%d"), 
                        'GAME_ID': int(game_ids[i]),                       
                        'SEASON': SEASON,
                        } 
        game_details_df = pd.DataFrame(game_details, index=[i])
        # append to new games dataframe
        df_today = pd.concat([df_today, game_details_df], ignore_index = True)

    #blank rows will be filled with 0 to prevent issues with feature engineering
    df_today = df_today.fillna(0) 

    df_today



**Access Feature Store**

In [9]:
project = hopsworks.login(api_key_value=HOPSWORKS_API_KEY)

# HOPSWORKS can be kinda buggy and has been throwing a lot of errors recently or even just failing to return data
# so I'm adding a try/except block to retry the query if it fails
tries = 5

for i in range(tries):
    
    try:
        fs = project.get_feature_store()
    except KeyError as e:
        if i < tries - 1: # i is zero indexed
            time.sleep(30)
            continue
        else:
            raise ValueError('HOPSWORKS failed to connect')
    break



Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/3350


Connected. Call `.close()` to terminate connection gracefully.


**Access Feature Group**

In [10]:
# HOPSWORKS can be kinda buggy and has been throwing a lot of errors recently or even just failing to return data
# so I'm adding a try/except block to retry the query if it fails
tries = 5

for i in range(tries):
    
    try:
        rolling_stats_fg = fs.get_feature_group(
        name="rolling_stats",
        version=2,
        )
    except KeyError as e:
        if i < tries - 1: # i is zero indexed
            time.sleep(30)
            continue
        else:
            raise ValueError('HOPSWORKS failed to connect')
    break



**Query Old Data Needed for Feature Engineering of New Data**

To generate features like rolling averages for the new games, older data from previous games is needed since some of the rolling averages might extend back 15 or 20 games or so.

In [11]:
BASE_FEATURES = ['game_date_est',
 'game_id',
 'home_team_id',
 'visitor_team_id',
 'season',
 'pts_home',
 'fg_pct_home',
 'ft_pct_home',
 'fg3_pct_home',
 'ast_home',
 'reb_home',
 'pts_away',
 'fg_pct_away',
 'ft_pct_away',
 'fg3_pct_away',
 'ast_away',
 'reb_away',
 'home_team_wins',
]

ds_query = rolling_stats_fg.select(BASE_FEATURES)

# HOPSWORKS can be kinda buggy and has been throwing a lot of errors recently or even just failing to return data
# so I'm adding a try/except block to retry the query if it fails
tries = 5

for i in range(tries):
    for j in range(tries):
        try:
            df_old = ds_query.read()
        except KeyError as e:
            if j < tries - 1: 
                time.sleep(10)
                continue
            else:
                raise ValueError('HOPSWORKS failed to connect')
        break

    if df_old.empty:
        if i < tries - 1: 
            time.sleep(10)
        else:
            raise ValueError('HOPSWORKS failed to return data')
    else:
        break



df_old


2023-05-01 06:36:01,703 INFO: USE `nba_predictor_featurestore`
2023-05-01 06:36:02,249 INFO: SELECT `fg0`.`game_date_est` `game_date_est`, `fg0`.`game_id` `game_id`, `fg0`.`home_team_id` `home_team_id`, `fg0`.`visitor_team_id` `visitor_team_id`, `fg0`.`season` `season`, `fg0`.`pts_home` `pts_home`, `fg0`.`fg_pct_home` `fg_pct_home`, `fg0`.`ft_pct_home` `ft_pct_home`, `fg0`.`fg3_pct_home` `fg3_pct_home`, `fg0`.`ast_home` `ast_home`, `fg0`.`reb_home` `reb_home`, `fg0`.`pts_away` `pts_away`, `fg0`.`fg_pct_away` `fg_pct_away`, `fg0`.`ft_pct_away` `ft_pct_away`, `fg0`.`fg3_pct_away` `fg3_pct_away`, `fg0`.`ast_away` `ast_away`, `fg0`.`reb_away` `reb_away`, `fg0`.`home_team_wins` `home_team_wins`
FROM `nba_predictor_featurestore`.`rolling_stats_2` `fg0`


,game_date_est,game_id,home_team_id,visitor_team_id,season,pts_home,fg_pct_home,ft_pct_home,fg3_pct_home,ast_home,reb_home,pts_away,fg_pct_away,ft_pct_away,fg3_pct_away,ast_away,reb_away,home_team_wins
0,2018-01-05,21700570,1610612738,1610612750,2017,91,0.404053,0.691895,0.166992,24,56,84,0.367920,0.750000,0.303955,15,43,1
1,2003-11-28,20300211,1610612738,1610612749,2003,106,0.525879,0.799805,0.420898,30,48,96,0.415039,0.724121,0.303955,21,40,1
2,2006-12-11,20600301,1610612753,1610612756,2006,89,0.451904,0.523926,0.166992,18,42,103,0.514160,0.852051,0.250000,25,36,0
3,2005-01-11,20400504,1610612760,1610612746,2004,104,0.437988,0.889160,0.500000,24,40,99,0.438965,0.758789,0.333008,23,47,1
4,2016-10-29,21600028,1610612752,1610612763,2016,111,0.525879,0.641113,0.285889,24,41,104,0.429932,0.808105,0.360107,19,40,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24045,2017-12-10,21700387,1610612754,1610612743,2017,126,0.526855,0.817871,0.399902,24,40,116,0.452881,0.812988,0.405029,24,47,1
24046,2015-04-10,21401175,1610612753,1610612761,2014,99,0.460938,1.000000,0.350098,21,46,101,0.415039,0.933105,0.300049,22,46,0
24047,2005-01-15,20400537,1610612745,1610612759,2004,73,0.333008,0.730957,0.399902,15,49,67,0.353027,0.629883,0.125000,10,39,1
24048,2012-03-07,21100575,1610612749,1610612741,2011,104,0.477051,0.789062,0.333008,29,35,106,0.518066,0.881836,0.312988,26,43,0


**Convert Feature Names back to original mixed case**

In [12]:
#hopsworks converts all feature names to lowercase, and for code reuse, we need to convert them back
df_old = convert_feature_names(df_old)
df_old
df_old[df_old['PTS_home'] == 0]

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
8127,2023-05-01,42200222,1610612743,1610612756,2022,0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0,0,0
16567,2023-05-01,42200211,1610612738,1610612755,2022,0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0,0,0


**Update Yesterday's Matchup Predictions with New Final Results**

In [13]:
# filter out games that are pending final results
# (these were the rows used for prediction yesterday)
# and then update these with the new results


# one approach is to simply drop the rows that were used for prediction yesterday
# which are games that have 0 points for home team
# and then append the new rows to the dataframe
df_old = df_old[df_old['PTS_home'] != 0]
df_old = pd.concat([df_old, df_new], ignore_index = True)

df_old

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,2018-01-05,21700570,1610612738,1610612750,2017,91,0.404053,0.691895,0.166992,24,56,84,0.367920,0.750000,0.303955,15,43,1
1,2003-11-28,20300211,1610612738,1610612749,2003,106,0.525879,0.799805,0.420898,30,48,96,0.415039,0.724121,0.303955,21,40,1
2,2006-12-11,20600301,1610612753,1610612756,2006,89,0.451904,0.523926,0.166992,18,42,103,0.514160,0.852051,0.250000,25,36,0
3,2005-01-11,20400504,1610612760,1610612746,2004,104,0.437988,0.889160,0.500000,24,40,99,0.438965,0.758789,0.333008,23,47,1
4,2016-10-29,21600028,1610612752,1610612763,2016,111,0.525879,0.641113,0.285889,24,41,104,0.429932,0.808105,0.360107,19,40,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24046,2012-03-07,21100575,1610612749,1610612741,2011,104,0.477051,0.789062,0.333008,29,35,106,0.518066,0.881836,0.312988,26,43,0
24047,2009-01-23,20800635,1610612750,1610612740,2008,116,0.513184,0.765137,0.476074,26,42,108,0.532227,0.846191,0.590820,23,26,1
24048,2023-04-30,42200201,1610612748,1610612752,2022,108,42.400000,79.300000,33.300000,26,39,101,47.700000,60.000000,20.600000,20,48,1
24049,2023-04-30,42200167,1610612744,1610612758,2022,120,43.000000,63.300000,32.600000,25,55,100,37.500000,59.300000,25.500000,21,49,1


**Add Today's Matchups for Feature Engineering**

In [14]:
if matchups is None:
    print('No games today')
    df_combined = df_old
else:
    df_combined = pd.concat([df_old, df_today], ignore_index = True)
    df_combined

**Data Processing**

In [15]:
df_combined = process_games(df_combined) 
df_combined = add_TARGET(df_combined)
df_combined

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS,PLAYOFF,TARGET
0,2018-01-05 00:00:00,21700570,1610612738,1610612750,2017,91.0,0.404053,0.691895,0.166992,24.0,56.0,84.0,0.367920,0.750000,0.303955,15.0,43.0,1.0,0,1.0
1,2003-11-28 00:00:00,20300211,1610612738,1610612749,2003,106.0,0.525879,0.799805,0.420898,30.0,48.0,96.0,0.415039,0.724121,0.303955,21.0,40.0,1.0,0,1.0
2,2006-12-11 00:00:00,20600301,1610612753,1610612756,2006,89.0,0.451904,0.523926,0.166992,18.0,42.0,103.0,0.514160,0.852051,0.250000,25.0,36.0,0.0,0,0.0
3,2005-01-11 00:00:00,20400504,1610612760,1610612746,2004,104.0,0.437988,0.889160,0.500000,24.0,40.0,99.0,0.438965,0.758789,0.333008,23.0,47.0,1.0,0,1.0
4,2016-10-29 00:00:00,21600028,1610612752,1610612763,2016,111.0,0.525879,0.641113,0.285889,24.0,41.0,104.0,0.429932,0.808105,0.360107,19.0,40.0,1.0,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24045,2005-01-15 00:00:00,20400537,1610612745,1610612759,2004,73.0,0.333008,0.730957,0.399902,15.0,49.0,67.0,0.353027,0.629883,0.125000,10.0,39.0,1.0,0,1.0
24046,2012-03-07 00:00:00,21100575,1610612749,1610612741,2011,104.0,0.477051,0.789062,0.333008,29.0,35.0,106.0,0.518066,0.881836,0.312988,26.0,43.0,0.0,0,0.0
24047,2009-01-23 00:00:00,20800635,1610612750,1610612740,2008,116.0,0.513184,0.765137,0.476074,26.0,42.0,108.0,0.532227,0.846191,0.590820,23.0,26.0,1.0,0,1.0
24051,2023-05-01,42200211,1610612738,1610612755,2022,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,1,0.0


**Feature Engineering**

In [16]:
# Feature engineering to add: 
    # rolling averages of key stats, 
    # win/lose streaks, 
    # home/away streaks, 
    # specific matchup (team X vs team Y) rolling averages and streaks

df_combined = process_features(df_combined)



#fix type conversion issues with hopsworks
df_combined['TARGET'] = df_combined['TARGET'].astype('int16')
df_combined['HOME_TEAM_WINS'] = df_combined['HOME_TEAM_WINS'].astype('int16')

df_combined


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#r

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,...,FG3_PCT_AVG_LAST_10_ALL_x_minus_y,FG3_PCT_AVG_LAST_15_ALL_x_minus_y,AST_AVG_LAST_3_ALL_x_minus_y,AST_AVG_LAST_7_ALL_x_minus_y,AST_AVG_LAST_10_ALL_x_minus_y,AST_AVG_LAST_15_ALL_x_minus_y,REB_AVG_LAST_3_ALL_x_minus_y,REB_AVG_LAST_7_ALL_x_minus_y,REB_AVG_LAST_10_ALL_x_minus_y,REB_AVG_LAST_15_ALL_x_minus_y
0,2003-10-28,20300001,1610612755,1610612748,2003,89,0.439941,0.533203,0.350098,25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2003-10-28,20300003,1610612747,1610612742,2003,109,0.505859,0.600098,0.350098,32,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2003-10-28,20300002,1610612759,1610612756,2003,83,0.425049,0.769043,0.099976,20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2003-10-29,20300006,1610612740,1610612737,2003,88,0.323975,0.700195,0.160034,24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2003-10-29,20300005,1610612752,1610612753,2003,83,0.399902,0.824219,0.312988,20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24045,2023-04-29,42200221,1610612756,1610612743,2022,107,51.187500,82.375000,30.406250,20,...,3.265625,3.184375,0.333333,0.428571,1.1,2.066667,1.000000,-1.142857,-1.2,0.133333
24046,2023-04-30,42200167,1610612744,1610612758,2022,120,43.000000,63.312500,32.593750,25,...,4.285938,4.545833,4.000000,7.714286,5.1,4.466667,-4.000000,1.285714,2.7,1.066667
24047,2023-04-30,42200201,1610612748,1610612752,2022,108,42.406250,79.312500,33.312500,26,...,7.848437,3.185417,3.666667,4.285714,3.6,0.733333,-1.333333,-2.000000,-2.0,-3.133333
24048,2023-05-01,42200222,1610612743,1610612756,2022,0,0.000000,0.000000,0.000000,0,...,-0.898438,-3.392708,-3.666667,-1.428571,-0.2,-1.600000,4.333333,3.571429,1.4,0.733333


**Insert New Data into Feature Group**

In [17]:
# HOPSWORKS can be kinda buggy and has been throwing a lot of errors recently or even just failing to return data
# so I'm adding a try/except block to retry the query if it fails
tries = 5

for i in range(tries):
    
    try:
        rolling_stats_fg.insert(df_combined, overwrite = True, write_options={"wait_for_job" : False})
    except KeyError as e:
        if i < tries - 1: 
            time.sleep(30)
            continue
        else:
            raise ValueError('HOPSWORKS failed to connect')
    break



Uploading Dataframe: 0.00% |          | Rows 0/24050 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/3350/jobs/named/rolling_stats_2_offline_fg_backfill/executions


In [18]:
df_combined

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,...,FG3_PCT_AVG_LAST_10_ALL_x_minus_y,FG3_PCT_AVG_LAST_15_ALL_x_minus_y,AST_AVG_LAST_3_ALL_x_minus_y,AST_AVG_LAST_7_ALL_x_minus_y,AST_AVG_LAST_10_ALL_x_minus_y,AST_AVG_LAST_15_ALL_x_minus_y,REB_AVG_LAST_3_ALL_x_minus_y,REB_AVG_LAST_7_ALL_x_minus_y,REB_AVG_LAST_10_ALL_x_minus_y,REB_AVG_LAST_15_ALL_x_minus_y
0,2003-10-28,20300001,1610612755,1610612748,2003,89,0.439941,0.533203,0.350098,25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2003-10-28,20300003,1610612747,1610612742,2003,109,0.505859,0.600098,0.350098,32,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2003-10-28,20300002,1610612759,1610612756,2003,83,0.425049,0.769043,0.099976,20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2003-10-29,20300006,1610612740,1610612737,2003,88,0.323975,0.700195,0.160034,24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2003-10-29,20300005,1610612752,1610612753,2003,83,0.399902,0.824219,0.312988,20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24045,2023-04-29,42200221,1610612756,1610612743,2022,107,51.187500,82.375000,30.406250,20,...,3.265625,3.184375,0.333333,0.428571,1.1,2.066667,1.000000,-1.142857,-1.2,0.133333
24046,2023-04-30,42200167,1610612744,1610612758,2022,120,43.000000,63.312500,32.593750,25,...,4.285938,4.545833,4.000000,7.714286,5.1,4.466667,-4.000000,1.285714,2.7,1.066667
24047,2023-04-30,42200201,1610612748,1610612752,2022,108,42.406250,79.312500,33.312500,26,...,7.848437,3.185417,3.666667,4.285714,3.6,0.733333,-1.333333,-2.000000,-2.0,-3.133333
24048,2023-05-01,42200222,1610612743,1610612756,2022,0,0.000000,0.000000,0.000000,0,...,-0.898438,-3.392708,-3.666667,-1.428571,-0.2,-1.600000,4.333333,3.571429,1.4,0.733333


In [19]:
# check to make sure there are no duplicate games were inadvertently added
df_combined[df_combined.duplicated(subset=['GAME_ID'], keep=False)]

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,...,FG3_PCT_AVG_LAST_10_ALL_x_minus_y,FG3_PCT_AVG_LAST_15_ALL_x_minus_y,AST_AVG_LAST_3_ALL_x_minus_y,AST_AVG_LAST_7_ALL_x_minus_y,AST_AVG_LAST_10_ALL_x_minus_y,AST_AVG_LAST_15_ALL_x_minus_y,REB_AVG_LAST_3_ALL_x_minus_y,REB_AVG_LAST_7_ALL_x_minus_y,REB_AVG_LAST_10_ALL_x_minus_y,REB_AVG_LAST_15_ALL_x_minus_y
